In [272]:
!pip3 install statsmodels
!pip3 install vaderSentiment
!pip3 install nbconvert
!pip3 install tabulate
!pip3 install --upgrade scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [273]:
import numpy as np
import pandas as pd

from IPython.display import display
from statistics import mean

from datetime import date, timedelta
import datetime

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.gof import chisquare as chisquare
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from statsmodels.stats.anova import anova_lm

import csv

from tabulate import tabulate

import scipy


In [274]:
current_date = datetime.datetime.strptime("2020-08-22", "%Y-%m-%d").date()

def convertToTime(row, columnName):
    return datetime.datetime.strptime(row[columnName], "%Y-%m-%d").date()

def convertToDate(row, columnName):
    return datetime.datetime.strptime(row[columnName], "%Y-%m-%d %H:%M:%S").date()
    
def update_end_date(row, columnName, latestDate):
    if row[columnName] == row[columnName]:
        return row[columnName]
    else:
        return latestDate

def getDays(row, beginColumnName, endColumnName):
    v = datetime.datetime.strptime(str(row[endColumnName]), "%Y-%m-%d").date() - datetime.datetime.strptime(row[beginColumnName], "%Y-%m-%d").date()
    return v.days

def calculate_recency(row, columnName):
    return (current_date - datetime.datetime.strptime(str(row[columnName]), "%Y-%m-%d").date()).days

def lookup_index(row, columnName, array):
    if(row[columnName] not in array):
        return -1
    return array.index(row[columnName]) + 1

def colour_life_events(row):
    colours = {'personal':'lightcoral', 'health':'orange', 'work':'lightgreen', 'financial':'teal', 'weather':'blueviolet', 'societal':'navy','other':'skyblue'}
    return colours[row['life_event_type']]

def remove_rows(base_df, other_df):
    modified_df = other_df.drop(other_df[other_df['snapshot_id'] not in base_df['snapshot_id'].values].index)
    return modified_df

def fix_signficance(row):
    if('significance' in row['valence']):
        return row['valence']
    else:
        return row['significance']

def fix_valence(row):
    if('significance' in row['significance']):
        return row['valence']
    else:
        return row['significance']  

def get_broad_category(row, categories, column_name):
    if(row[column_name] in categories):
        return categories[row[column_name]]
    return "UNKNOWN"
    
def compute_sentiment(row):
    post = row['Text']
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(post)
    sentiment = 0
    if (vs["neu"]>0.8):
        sentiment = 0
    elif (vs["pos"]==vs["neg"]):
        sentiment = 0
    elif (vs["pos"]>vs["neg"]):
        sentiment = 1
    elif (vs["neg"]>vs["neu"]):
        sentiment = -1
    return sentiment

def convert_valence_to_sentiment(row):
    valence = row['valence']
    retVal = 0
    if (valence == 'Neither Positive or Negative'):
        retVal = 0
    elif("Positive" in valence):
        retVal = 1
    elif("Negative" in valence):
        retVal = -1
    return retVal

def update_status(row, columnName):
    if row[columnName] == row[columnName]:
        if "ongoing" in row[columnName].lower():
            return "Ongoing"
        return row[columnName]
    else:
        return "Ended"

def update_education_level(row, columnName):
    if 'college' in row[columnName].lower():
        return "College Degree"
    elif 'doctoral' in row[columnName].lower():
        return "Doctoral Degree"
    elif 'master' in row[columnName].lower() or 'grad' in row[columnName].lower():
        return "Graduate Degree"
    elif 'hs' in row[columnName].lower() or 'high school' in row[columnName].lower():
        return "High School"
    
    return row[columnName]

def add_avg_dependent_variable(row, weekly_data):
    start_date = datetime.datetime.strptime(str(row['UpdatedBeginDate']), "%Y-%m-%d").date()
    end_date = datetime.datetime.strptime(str(row['UpdatedEndDate']), "%Y-%m-%d").date()
    year_week = []
    delta = timedelta(days=1)
    dependent_variable_data = []
    while(start_date < end_date):
        (year, week) = (start_date.isocalendar()[0], start_date.isocalendar()[1])
        if( (year, week) not in year_week):
            year_week.append((year, week))
            if (year, week) in weekly_data[row['snapshot_id']]:
                dependent_variable_data.append(weekly_data[row['snapshot_id']][(year, week)])
        start_date = start_date + delta
    if(len(dependent_variable_data) > 0):
        return mean(dependent_variable_data)
    else:
        return -1
    

def add_avg_date_based_dependent_variable(row, weekly_data):
    date = datetime.datetime.strptime(str(row['created_date']), "%Y-%m-%d").date()
    year, week = (date.isocalendar()[0], date.isocalendar()[1])
    
    if (year,week) in weekly_data[row['snapshot_id']]:
        return weekly_data[row['snapshot_id']][(year, week)]
    else:
        return -1

In [275]:
control_variables             = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender'
combined_control_variables    = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + data_type'

sr_life_event_variables       = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_life_event_variables       = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_life_event_variables = 'Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'

sr_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + significance_label'
sm_all_variables     = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope'
combined_all_variables        = 'shipley_vocab + shipley_abs + openness + conscientiousness + extraversion + agreeableness + neuroticism + pos_affect + neg_affect + stai_trait + education_level + psqi + age + gender + Anticipation + LifeEventFamily + valence + recency + status + Intimacy + Scope + data_type'
# combined_sm_sr_all_variables  = 'shipley_vocab_sm + shipley_vocab_sr + shipley_abs_sm + shipley_abs_sr + openness_sm + openness_sr + conscientiousness_sm + conscientiousness_sr + extraversion_sm + extraversion_sr + agreeableness_sm + agreeableness_sr + neuroticism_sm + neuroticism_sr + pos_affect_sm + pos_affect_sr + neg_affect_sm + neg_affect_sr + stai_trait_sm + stai_trait_sr + education_level_sm + education_level_sr + psqi_sm + psqi_sr + age_sm + age_sr + gender_sm + gender_sr + Anticipation_sm + Anticipation_sr + LifeEventFamily_sm + LifeEventFamily_sr + valence_sm + valence_sr + Intimacy_sm + Intimacy_sr + Scope_sm + Scope_sr + data_type'

In [276]:
def load_demographics_data():
    demographics_data = pd.read_csv('data/igtbs_demographics_complete.csv', parse_dates=True)
    demographics_data = demographics_data[['age','gender','snapshot_id', 'shipley.vocab', 'shipley.abs', 'openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism','pos.affect','neg.affect','stai.trait','psqi','educ']]
    demographics_data['education_level'] = demographics_data.apply(update_education_level, columnName='educ', axis=1)
    demographics_data = demographics_data.drop(columns=['educ'])
    demographics_data = demographics_data.rename(columns={
        'shipley.vocab': 'shipley_vocab',
        'shipley.abs': 'shipley_abs',
        'pos.affect': 'pos_affect',
        'neg.affect': 'neg_affect',
        'stai.trait': 'stai_trait'
    })
    return demographics_data

In [277]:
load_demographics_data()

,age,gender,snapshot_id,shipley_vocab,shipley_abs,openness,conscientiousness,extraversion,agreeableness,neuroticism,pos_affect,neg_affect,stai_trait,psqi,education_level
0,39.0,Female,11156103277197680506,32.0,15.0,4.750000,4.666667,3.666667,4.416667,2.333333,38.0,15.0,39.0,7.0,College Degree
1,39.0,Female,13346780717560972831,38.0,19.0,3.500000,4.833333,3.000000,4.916667,1.833333,41.0,16.0,28.0,8.0,Graduate Degree
2,25.0,Female,12932409338132389043,32.0,19.0,4.916667,4.083333,3.750000,4.000000,4.083333,20.0,24.0,55.0,10.0,Graduate Degree
3,33.0,Male,10385244076367909694,34.0,17.0,3.416667,2.166667,2.916667,3.000000,2.000000,25.0,24.0,51.0,9.0,Graduate Degree
4,37.0,Female,10671464268435700062,39.0,15.0,4.500000,5.000000,3.916667,3.750000,3.000000,36.0,16.0,35.0,3.0,College Degree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,44.0,Male,10033784408945437200,34.0,17.0,3.500000,4.250000,2.750000,4.500000,2.166667,30.0,13.0,35.0,4.0,Graduate Degree
750,51.0,Male,13257691313989387573,34.0,20.0,4.500000,2.666667,2.750000,4.250000,2.833333,35.0,17.0,41.0,5.0,Graduate Degree
751,44.0,Female,13370808519384713343,35.0,18.0,3.500000,4.250000,4.333333,3.250000,1.083333,39.0,12.0,25.0,8.0,Graduate Degree
752,31.0,Female,12173290729829297463,32.0,17.0,3.583333,4.666667,2.916667,3.416667,3.166667,38.0,18.0,31.0,3.0,College Degree


In [278]:
def load_survey_categories():
    df_self_reported_categories = pd.read_csv('data/Life Events Categories Mapping - Self-Reported Categories.csv')
    return df_self_reported_categories

def load_survey_data_without_categories():
    df_survey = pd.read_csv('data/Superimposed/LifeEvents_Curated_non_blinded.csv', parse_dates=True)    
    df_survey = df_survey[['snapshot_id', 'description','UpdatedBeginDate', 'UpdatedEndDate','life_event_type', 'work_perf_impact', 'significance','valence', 'ended_or_ongoing']]

    # Date manipulation
    latest_date = max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedEndDate'])
    latest_date = max(latest_date, max(datetime.datetime.strptime(str(x), "%Y-%m-%d").date() if x == x else datetime.date.min for x in df_survey['UpdatedBeginDate']))
    df_survey = df_survey.drop(df_survey[df_survey['UpdatedBeginDate'].isnull() == True].index)
    df_survey['UpdatedEndDate'] = df_survey.apply(update_end_date, columnName='UpdatedEndDate', latestDate=latest_date, axis=1)
    df_survey['num_of_days'] = df_survey.apply(getDays, endColumnName='UpdatedEndDate', beginColumnName='UpdatedBeginDate', axis=1)
    df_survey['UpdatedBeginDate_time'] = df_survey.apply(convertToTime, columnName='UpdatedBeginDate', axis=1)
    df_survey['recency'] = df_survey.apply(calculate_recency, columnName='UpdatedEndDate', axis=1)

    # Update values for valence and significance
    df_survey.replace({'valence': {np.nan: 'Neither Positive or Negative'}, 'significance': {np.nan: 'Neither Positive or Negative'}}, inplace=True)
    df_survey['fixed_signficance'] = df_survey.apply(fix_signficance, axis = 1)
    df_survey['fixed_valence'] = df_survey.apply(fix_valence, axis = 1)
    df_survey = df_survey.drop(columns = ['valence', 'significance'])
    df_survey = df_survey.rename(columns={"fixed_signficance": "significance", "fixed_valence": "valence"})
    df_survey['valence'] = df_survey.apply(convert_valence_to_sentiment, axis=1)
    df_survey['ended_or_ongoing'] = df_survey.apply(update_status, columnName='ended_or_ongoing', axis=1)

    # Select columns we are interested in
    df_survey = df_survey[['snapshot_id', 'description', 'UpdatedBeginDate', 'UpdatedEndDate', 'significance', 'valence', 'ended_or_ongoing', 'recency']]

    # Label encoding for significance
    le_significance = LabelEncoder()
    le_significance.fit(df_survey['significance'].values)
    df_survey['significance_label'] = df_survey.apply(lambda x: le_significance.transform([x['significance']])[0], axis=1)
    df_survey = df_survey.drop(columns=['significance'])

    return df_survey

def load_survey_data():
    df_survey_without_categories = load_survey_data_without_categories()
    df_self_reported_categories = load_survey_categories()
    df_survey = pd.merge(df_survey_without_categories, df_self_reported_categories, how="inner", left_on="description", right_on="SR_LifeEvent")
    df_survey = df_survey.drop(columns=['description', 'SR_LifeEvent', 'LifeEventFinal', 'LifeEventFamily2'])
    df_survey = df_survey.rename(columns={'ended_or_ongoing':'status'})

    return df_survey

In [279]:
def load_social_media_categories():
    df_social_media_categories = pd.read_csv('data/Life Events Categories Mapping - Social Media Categories-2.csv')
    return df_social_media_categories

def load_social_media_data_without_categories():
    df_social_media_data = pd.read_csv('data/Superimposed/Facebook Data For Life Events-Combined - FB Data.csv')
    df_social_media_data = df_social_media_data[['snapshot_id', 'created_time', 'Text', 'final_life_event_category_2', 'ended/ongoing']]
    df_social_media_data = df_social_media_data.replace({'PostiveMove':'PositiveMove', 'Negative Move':'NegativeMove'})
    df_social_media_data = df_social_media_data.drop(df_social_media_data[((df_social_media_data['final_life_event_category_2'].isnull() == True))].index)
    df_social_media_data['created_date'] = df_social_media_data.apply(convertToDate, columnName='created_time', axis=1)
    df_social_media_data['valence'] = df_social_media_data.apply(compute_sentiment, axis=1)
    df_social_media_data = df_social_media_data.drop(columns=['created_time','Text'])
    df_social_media_data['recency'] = df_social_media_data.apply(calculate_recency, columnName='created_date', axis=1)
    df_social_media_data['ended/ongoing'] = df_social_media_data.apply(update_status, columnName='ended/ongoing', axis=1)
    return df_social_media_data

def load_social_media_data():
    df_social_media_data = load_social_media_data_without_categories()
    df_social_media_categories = load_social_media_categories()
    df_social_media_data_with_categories = pd.merge(df_social_media_data, df_social_media_categories, how="inner", left_on='final_life_event_category_2', right_on='SM_LifeEvent')
    df_social_media_data_with_categories = df_social_media_data_with_categories.drop(columns=['final_life_event_category_2','SM_LifeEvent','LifeEventFamily2','Comments','SignificanceRank'])
    df_social_media_data_with_categories = df_social_media_data_with_categories.rename(columns={'ended/ongoing':'status'})
    return df_social_media_data_with_categories

In [280]:
load_social_media_data()

,snapshot_id,status,created_date,valence,recency,LifeEventFamily,Anticipation,Intimacy,Scope
0,12840161237957324034,Ongoing,2018-03-16,0,890,Personal,Anticipated,2,2
1,11156103277197680506,Ongoing,2018-07-31,1,753,Personal,Anticipated,2,2
2,11329586216091316598,Ongoing,2018-09-08,0,714,Personal,Anticipated,2,2
3,13543308681200334026,Ongoing,2018-07-22,0,762,Personal,Anticipated,2,2
4,12676581714475000627,Ongoing,2018-04-25,0,850,Personal,Anticipated,2,2
...,...,...,...,...,...,...,...,...,...
1975,10729524556661174446,Ongoing,2018-12-19,0,612,School,Unanticipated,3,2
1976,9767765376628075157,Ongoing,2018-07-27,0,757,Health,Anticipated,3,3
1977,9767765376628075157,Ongoing,2018-10-23,0,669,Health,Anticipated,3,3
1978,9767765376628075157,ended,2018-09-19,1,703,Health,Unanticipated,3,3


In [299]:
def load_dailies_data():
    df_dailies = pd.read_csv('data/Superimposed/dailies_scores.csv', low_memory=False)
    df_dailies = df_dailies[['snapshot_id','day', 'alc_status', 'alc.quantity.d', 'anxiety.d', 'pos.affect.d', 'neg.affect.d','sleep.d', 'stress.d']]
    df_dailies['day_time'] = df_dailies.apply(convertToTime, columnName='day', axis=1)
    df_dailies = df_dailies.rename(columns={'alc.quantity.d': 'alc_quantity',
    'anxiety.d': 'anxiety',
    'pos.affect.d': 'pos_affect',
    'neg.affect.d': 'neg_affect',
    'sleep.d': 'sleep',
    'stress.d': 'stress'})
    df_dailies['sleep'] = df_dailies['sleep'] + 1
    return df_dailies

In [282]:
load_dailies_data()

,snapshot_id,day,alc_status,alc_quantity,anxiety,pos_affect,neg_affect,sleep,stress,day_time
0,10003528337325499062,2018-03-01,yes,3.0,2.0,11.0,6.0,6.0,2.0,2018-03-01
1,10003528337325499062,2018-03-02,no,0.0,2.0,13.0,5.0,8.0,2.0,2018-03-02
2,10003528337325499062,2018-03-04,no,0.0,1.0,5.0,5.0,9.0,1.0,2018-03-04
3,10003528337325499062,2018-03-06,no,0.0,1.0,15.0,5.0,8.0,1.0,2018-03-06
4,10003528337325499062,2018-03-09,no,0.0,2.0,11.0,5.0,8.0,1.0,2018-03-09
...,...,...,...,...,...,...,...,...,...,...
38592,9999689421615292586,2018-06-23,NaN,NaN,1.0,8.0,5.0,NaN,1.0,2018-06-23
38593,9999689421615292586,2018-06-25,NaN,NaN,1.0,5.0,5.0,NaN,1.0,2018-06-25
38594,9999689421615292586,2018-06-28,NaN,NaN,2.0,12.0,5.0,NaN,2.0,2018-06-28
38595,9999689421615292586,2018-06-30,NaN,NaN,1.0,8.0,5.0,NaN,1.0,2018-06-30


In [283]:
def get_average_values(df_dailies, merged_data, start_date_column, dependent_variable):
    snapshot_id_day_value = {}
    for index, row in df_dailies.iterrows():
        if row['snapshot_id'] not in snapshot_id_day_value:
            snapshot_id_day_value[row['snapshot_id']] = {}
        snapshot_id_day_value[row['snapshot_id']][str(row['day'])] = row[dependent_variable]

    merged_data[dependent_variable] = -1
    delta_5 = timedelta(days=5)
    delta_3 = timedelta(days=3)
    delta_7 = timedelta(days=7)
    one_day = timedelta(days=1)
    week_avg = []
    for index, row in merged_data.iterrows():
        if row['snapshot_id'] in snapshot_id_day_value:
            if row['Anticipation'] == 'Anticipated':
                start_date = datetime.datetime.strptime(str(row[start_date_column]), "%Y-%m-%d").date() - delta_7
                end_date = datetime.datetime.strptime(str(row[start_date_column]), "%Y-%m-%d").date() + delta_5
            else:
                start_date = datetime.datetime.strptime(str(row[start_date_column]), "%Y-%m-%d").date() - delta_5
                end_date = datetime.datetime.strptime(str(row[start_date_column]), "%Y-%m-%d").date() + delta_7
            value = 0
            count = 0
            while start_date <= end_date:
                if (str(start_date) in snapshot_id_day_value[row['snapshot_id']]):
                    if snapshot_id_day_value[row['snapshot_id']][str(start_date)] == snapshot_id_day_value[row['snapshot_id']][str(start_date)]:
                        value+=snapshot_id_day_value[row['snapshot_id']][str(start_date)]
                        count+=1
                start_date+=one_day
            if count > 0:
                row[dependent_variable] = round(value / count, 3)
                week_avg.append(row.copy(deep=True))
    return week_avg

In [284]:
def build_df_for_regression_baseline(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/WeekRangeAnticipated/baseline_regression_'+dependent_variable+'.csv')
    except:
        df_demographics = load_demographics_data()
        df_dailies = load_dailies_data()
        
        snapshot_id_day_value = {}
        for index, row in df_dailies.iterrows():
            if row[dependent_variable] != row[dependent_variable]:
                continue
            if row['snapshot_id'] not in snapshot_id_day_value:
                snapshot_id_day_value[row['snapshot_id']] = {}
            snapshot_id_day_value[row['snapshot_id']][str(row['day'])] = row[dependent_variable]
            
        df_dailies = df_dailies[[dependent_variable, 'day', 'snapshot_id']]
        merged_data = pd.merge(df_dailies, df_demographics, how="inner", on=["snapshot_id"])

        week_avg = []
        merged_data[dependent_variable] = -1
        before_delta = timedelta(days=7)
        delta = timedelta(days=7)
        one_day = timedelta(days=1)
        
        for index, row in merged_data.iterrows():
            if row['snapshot_id'] in snapshot_id_day_value:

                start_date = datetime.datetime.strptime(str(row['day']), "%Y-%m-%d").date() - before_delta
                end_date = datetime.datetime.strptime(str(row['day']), "%Y-%m-%d").date() + delta
                value = 0
                count = 0
                while start_date <= end_date:
                    if (str(start_date) in snapshot_id_day_value[row['snapshot_id']]):
                        value+=snapshot_id_day_value[row['snapshot_id']][str(start_date)]
                        count+=1
                    start_date+=one_day

                if count > 0:
                    row[dependent_variable] = round(value / count, 3)
                    week_avg.append(row.copy(deep=True))

        merged_data = pd.DataFrame(week_avg, columns=merged_data.columns)
        merged_data.reset_index(drop=True, inplace=True)
        
        merged_data.to_csv('Linear Regression/WeekRangeAnticipated/baseline_regression_'+dependent_variable+'.csv', index=False)

    merged_data = merged_data.drop(columns=['day', 'snapshot_id'])
    merged_data = merged_data.dropna()
    return merged_data

In [303]:
def build_df_for_regression_survey_average_weekly(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/WeekRangeAnticipated/linear_regression_survey_weekly_'+dependent_variable+'.csv')
    except:
        df_survey = load_survey_data()
        df_demographics = load_demographics_data()
        df_dailies = load_dailies_data()
        merged_data = pd.merge(df_survey, df_demographics, how="inner", on=["snapshot_id"])

        week_avg = get_average_values(df_dailies, merged_data, 'UpdatedBeginDate', dependent_variable)

        merged_data = pd.DataFrame(week_avg, columns=merged_data.columns)
        merged_data.reset_index(drop=True, inplace=True)

        merged_data.to_csv('Linear Regression/WeekRangeAnticipated/linear_regression_survey_weekly_'+dependent_variable+'.csv', index=False)
    merged_data = merged_data.drop(merged_data[merged_data['education_level'] == 'High School'].index)   
    merged_data = merged_data.drop(merged_data[merged_data['LifeEventFamily'] == 'School'].index)

    merged_data = merged_data.drop(columns=[ 'UpdatedBeginDate', 'UpdatedEndDate', 'snapshot_id'])
    merged_data = merged_data.dropna()
        
    return merged_data

In [304]:
def build_df_for_regression_social_media_average_weekly(dependent_variable):
    try:
        merged_data = pd.read_csv('Linear Regression/WeekRangeAnticipated/linear_regression_social_media_weekly_'+dependent_variable+'.csv')
    except:
        df_social_media = load_social_media_data()
        df_demographics = load_demographics_data()
        df_dailies = load_dailies_data()
        merged_data = pd.merge(df_social_media, df_demographics, how="inner", on=["snapshot_id"])
        
        week_avg = get_average_values(df_dailies, merged_data, 'created_date', dependent_variable)

        merged_data = pd.DataFrame(week_avg, columns=merged_data.columns)
        merged_data.reset_index(drop=True, inplace=True)
        
        merged_data.to_csv('Linear Regression/WeekRangeAnticipated/linear_regression_social_media_weekly_'+dependent_variable+'.csv', index=False)
    
    merged_data = merged_data.drop(merged_data[merged_data['education_level'] == 'High School'].index)
    merged_data = merged_data.drop(merged_data[merged_data['LifeEventFamily'] == 'School'].index)
    merged_data = merged_data.drop(columns=['snapshot_id', 'created_date'])
    merged_data = merged_data.dropna()
        
    return merged_data

In [288]:
def build_df_for_regression_combined_survey_social_media(dependent_variable):
    df_social_media = build_df_for_regression_social_media_average_weekly(dependent_variable)
    df_social_media['data_type'] = 'Social Media'

    df_survey = build_df_for_regression_survey_average_weekly(dependent_variable)
    df_survey['data_type'] = 'Survey'
    df_survey = df_survey.drop(columns=['significance_label'])

    X_input = pd.concat([df_survey, df_social_media])
    X_input = X_input.dropna()
    return X_input

In [289]:
def regression(X_train, dependent_variable, formula):
    mod = smf.ols(formula=dependent_variable+'~'+formula, data=X_train)
    res = mod.fit()
    return res

## Run Regression  

In [290]:
def baseline_control_variables(dependent_variable, split=0):
    X_input = build_df_for_regression_baseline(dependent_variable)

    if split == 0:
        return regression(X_input, dependent_variable, control_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, control_variables)
        return X_test, model

In [291]:
def life_event_variables_all_data(dependent_variable, split=0):
    X_input = build_df_for_regression_combined_survey_social_media(dependent_variable)
    
    if split == 0:
        return regression(X_input, dependent_variable, combined_life_event_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, combined_life_event_variables)
        return X_test, model

In [292]:
def life_event_variables_sr(dependent_variable, split=0):
    X_input = build_df_for_regression_survey_average_weekly(dependent_variable)

    if split == 0:
        return regression(X_input, dependent_variable, sr_life_event_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, sr_life_event_variables)
        return X_test, model

In [293]:
def life_event_variables_sm(dependent_variable, split=0):
    X_input = build_df_for_regression_social_media_average_weekly(dependent_variable)

    if split == 0:
        return regression(X_input, dependent_variable, sm_life_event_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, sm_life_event_variables)
        return X_test, model

In [294]:
def all_variables_all_data(dependent_variable, split=0):
    X_input = build_df_for_regression_combined_survey_social_media(dependent_variable)

    if split == 0:
        return regression(X_input, dependent_variable, combined_all_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, combined_all_variables)
        return X_test, model

In [295]:
def all_variables_sr(dependent_variable, split=0):
    X_input = build_df_for_regression_survey_average_weekly(dependent_variable)

    if split == 0:
        return regression(X_input, dependent_variable, sr_all_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, sr_all_variables)
        return X_test, model

In [296]:
def all_variables_sm(dependent_variable, split=0):
    X_input = build_df_for_regression_social_media_average_weekly(dependent_variable)

    if split == 0:
        return regression(X_input, dependent_variable, sm_all_variables)
    else:
        X_train, X_test = train_test_split(X_input, test_size=split, random_state=85)
        model = regression(X_train, dependent_variable, sm_all_variables)
        return X_test, model

# Regression Results

In [309]:
def run_baseline_fn(fn, dependent_variable):
    model = fn(dependent_variable)
    dataset, train_model = fn(dependent_variable, 0.20)

    f_obs = train_model.predict(dataset).values
    f_exp = dataset[dependent_variable].values

    pearson = scipy.stats.pearsonr(f_obs, f_exp)
    return model, [round(model.rsquared_adj, 3), round(pearson[0], 3), "-", "-"]

def run_regression_fn(baseline, fn, dependent_variable):
    model = fn(dependent_variable)

    dataset, train_model = fn(dependent_variable, 0.20)

    f_obs = train_model.predict(dataset).values
    f_exp = dataset[dependent_variable].values

    pearson = scipy.stats.pearsonr(f_obs, f_exp)
    variance = anova_lm(baseline, model)
    return [round(model.rsquared_adj, 3), round(pearson[0], 3), round(variance.F.values[-1], 3), round(variance['Pr(>F)'].values[-1], 3)]


def run_all_regression(dependent_variable):
    data = []
    baseline_model, results = run_baseline_fn(baseline_control_variables, dependent_variable)
    data.append(["Dailies", "Control"] + results)
    
    data.append(["Social Media + Survey", "Life Event"] + run_regression_fn(baseline_model, life_event_variables_all_data, dependent_variable))
    data.append(["Social Media + Survey", "Control + Life Event"] + run_regression_fn(baseline_model, all_variables_all_data, dependent_variable))
    
    data.append(["Social Media", "Life Event"] + run_regression_fn(baseline_model, life_event_variables_sm, dependent_variable))
    data.append(["Social Media", "Control + Life Event"] + run_regression_fn(baseline_model, all_variables_sm, dependent_variable))

    data.append(["Survey", "Life Event"] + run_regression_fn(baseline_model, life_event_variables_sr, dependent_variable))
    data.append(["Survey", "Control + Life Event"] + run_regression_fn(baseline_model, all_variables_sr, dependent_variable))

    return data

### STRESS

In [310]:
d = run_all_regression('stress')
regression_results = pd.DataFrame(d, columns=["Dataset", "Variables", "Adjusted R-square", "Pearson Correlation", "F-statistic", "P-value"])
regression_results

,Dataset,Variables,Adjusted R-square,Pearson Correlation,F-statistic,P-value
0,Dailies,Control,0.180,0.438,-,-
1,Social Media + Survey,Life Event,0.067,0.257,0.791,1
2,Social Media + Survey,Control + Life Event,0.210,0.434,0.938,0.903
3,Social Media,Life Event,0.077,0.282,0.798,1
4,Social Media,Control + Life Event,0.223,0.483,0.952,0.816
5,Survey,Life Event,0.004,0.056,0.765,0.994
6,Survey,Control + Life Event,0.262,0.129,1.034,0.406


### SLEEP 

In [311]:
d = run_all_regression('sleep')
regression_results = pd.DataFrame(d, columns=["Dataset", "Variables", "Adjusted R-square", "Pearson Correlation", "F-statistic", "P-value"])
regression_results

,Dataset,Variables,Adjusted R-square,Pearson Correlation,F-statistic,P-value
0,Dailies,Control,0.078,0.272,-,-
1,Social Media + Survey,Life Event,0.003,0.072,0.486,1
2,Social Media + Survey,Control + Life Event,0.108,0.316,0.546,1
3,Social Media,Life Event,0.002,0.244,0.472,1
4,Social Media,Control + Life Event,0.144,0.411,0.553,1
5,Survey,Life Event,-0.003,0.003,0.557,1
6,Survey,Control + Life Event,0.117,0.275,0.634,1


### ANXIETY 

In [312]:
d = run_all_regression('anxiety')
regression_results = pd.DataFrame(d, columns=["Dataset", "Variables", "Adjusted R-square", "Pearson Correlation", "F-statistic", "P-value"])
regression_results

,Dataset,Variables,Adjusted R-square,Pearson Correlation,F-statistic,P-value
0,Dailies,Control,0.215,0.476,-,-
1,Social Media + Survey,Life Event,0.071,0.268,0.728,1
2,Social Media + Survey,Control + Life Event,0.238,0.474,0.893,0.99
3,Social Media,Life Event,0.086,0.365,0.764,1
4,Social Media,Control + Life Event,0.240,0.516,0.922,0.93
5,Survey,Life Event,0.018,0.005,0.636,1
6,Survey,Control + Life Event,0.323,0.376,0.925,0.761
